# Prepare data for stock study

## Data informaiton
- Technical.csv: contains stock trading information for every trade-day since 2011-01-04
- Fundamental.csv: contains other more general information for each stock and it is quatily level


In [2]:
import pandas as pd
import numpy as np

technical = pd.read_csv('/Users/jiaqizhang/Documents/innovation_day/stock/data/Technical.csv')
technical['transaction_date'] = pd.to_datetime(technical['Tradedate'])
technical['ipo_date'] = pd.to_datetime(technical['ipodate'], format="%Y/%m/%d")
technical = technical.drop(['Tradedate','ipodate','Unnamed: 13'],1)
technical['weekday'] = technical.transaction_date.dt.dayofweek
technical['out_date'] = technical.transaction_date + pd.Timedelta('7 days')
technical_r = technical.drop(['turn', 'mkt_cap', 'vol', 'adjfactor', 'amount', 
                              'swing','high', 'low','out_date','ipo_date','weekday'], 1)

technical_r = technical_r.rename(columns = {'transaction_date':'out_date'})

technical_tag = technical.join(technical_r.set_index(['code', 'out_date']), 
                               on=['code', 'out_date'], rsuffix='_7d')
technical_tag['closed_diff'] = (technical_tag['close'] - technical_tag['close_7d']) / technical_tag['close']

tech_key = technical_tag[['code','transaction_date']]


In [4]:
fund = pd.read_csv('/Users/jiaqizhang/Documents/innovation_day/stock/data/Fundamental.csv')
fund['issue_date'] = pd.to_datetime(fund['issuedate'])
fund = fund.drop(['reportdate', 'issuedate', 'ipodate', 'Unnamed: 61'], 1)
fund_key = fund[['code','issue_date']]

In [5]:

base_key = tech_key.join(fund_key.set_index(['code']), on = 'code')
base_key_f = base_key[ base_key.transaction_date > base_key.issue_date ]
base_key_m = base_key_f.groupby(['code', 'transaction_date'], as_index=False)['issue_date'].max()

base_0 = pd.merge(technical_tag, base_key_m, on=['code', 'transaction_date'], how='left')
base = base_0.merge(fund, how='left', on=['code', 'issue_date'], suffixes=('','_f') )






In [11]:
var_list = base.dtypes

var_info = pd.DataFrame(var_list)

var_info.columns = ['dtype']
var_info['var_name'] = var_info.index

var_info['type'] = 'variables'

meta_variables = ['code', 'transaction_date', 'ipo_date', 'out_date', 'close_7d', 'open_7d', 'closed_diff', 'issue_date']

base['days_since_ipo'] = (base['transaction_date'] - base['ipo_date']) / np.timedelta64(1, 'D')

base.to_csv('/Users/jiaqizhang/Documents/innovation_day/stock/data/base.csv')




In [13]:
store = pd.HDFStore('/Users/jiaqizhang/Documents/innovation_day/stock/data/base_store.h5')
store['base'] = base

In [14]:

store.close()
